In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split


In [3]:
#df_movies = pd.read_csv("Data/movies.csv")
df_movies = pd.read_csv("Data/movies.csv")
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
#df_ratings = pd.read_csv("/content/sample_data/ratings.csv")
df_ratings = pd.read_csv("Data/ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:

df_ratings.drop(columns='timestamp', inplace=True)

In [6]:
df_cleaned = df_movies.merge(df_ratings, on='movieId')

In [7]:
df_cleaned['num_viewers'] = df_cleaned.groupby('movieId')['userId'].transform('count')

In [8]:
df_clean = df_cleaned.drop_duplicates(subset='title', keep='first')
df_clean.head()


,movieId,title,genres,userId,rating,num_viewers
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215
215,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,110
325,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,52
377,4,Waiting to Exhale (1995),Comedy|Drama|Romance,6,3.0,7
384,5,Father of the Bride Part II (1995),Comedy,6,5.0,49


In [9]:
genres_split = df_clean.genres.apply(lambda x: x.split(sep='|')).apply(pd.value_counts, 1.0).fillna(0.0)
df_clean = pd.concat([df_clean.iloc[:,:], genres_split], axis=1)
df_clean.head()

,movieId,title,genres,userId,rating,num_viewers,Children,Comedy,Adventure,Animation,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,110,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,52,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
377,4,Waiting to Exhale (1995),Comedy|Drama|Romance,6,3.0,7,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
384,5,Father of the Bride Part II (1995),Comedy,6,5.0,49,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_clean['(no genres listed)'].sum()

34.0

In [11]:
df_clean.drop(columns=['(no genres listed)'], inplace=True)

# Hybrid Model

In [13]:
import pandas as pd
import surprise
from surprise import accuracy
from surprise import Dataset, Reader, SVD, KNNWithMeans
from surprise.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors


In [14]:
# create a reader to read the dataframe
reader = Reader(rating_scale=(1.0, 5.0))

# create a surprise dataset from the dataframe
data = Dataset.load_from_df(df_clean[['userId', 'movieId', 'rating']], reader)

# split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# train a content-based model using the KNN algorithm
k = 10
sim_options = {'name': 'cosine', 'user_based': False}
algo_cb = KNNWithMeans(k=k, sim_options=sim_options)
algo_cb.fit(trainset)

# train a collaborative filtering model using SVD
algo_cf = SVD()
algo_cf.fit(trainset)

# for each user and item pair in the test set, make predictions using both models
predictions_cf = algo_cf.test(testset)
predictions_cb = algo_cb.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
# assign weights to the models
weight_cf = 0.5
weight_cb = 0.5

# combine the predictions from both models using a weighted average
predictions_combined = []

# iterate over each prediction in the test set
for pred_cf, pred_cb in zip(predictions_cf, predictions_cb):
    # extract the user ID and item ID from the prediction
    uid, iid = pred_cf.uid, pred_cf.iid
    
    # calculate the weighted sum of the predicted ratings
    rating_combined = (weight_cf * pred_cf.est) + (weight_cb * pred_cb.est)
    
    # create a new prediction object with the combined rating
    pred_combined = surprise.prediction_algorithms.predictions.Prediction(uid, iid, r_ui=None, est=rating_combined, details=None)
    
    # add the combined prediction to the list
    predictions_combined.append(pred_combined)

In [16]:
##make predictions for all the movies that the user has not yet rated using the hybrid model

# get the list of all movie IDs that the user has not yet rated
user_id = 1  # replace with the user ID you want to get recommendations for
movie_ids = df_clean[~df_clean['movieId'].isin(df_clean[df_clean['userId'] == user_id]['movieId'])]['movieId']

# create a list of (user_id, movie_id, 0) tuples to make predictions on
testset = [[user_id, movie_id, 0] for movie_id in movie_ids]

# make predictions using both models
predictions_cf = algo_cf.test(testset)
predictions_cb = algo_cb.test(testset)

# combine the predictions from both models using a weighted average
predictions_combined = [(weight_cf * pred_cf.est) + (weight_cb * pred_cb.est) for pred_cf, pred_cb in zip(predictions_cf, predictions_cb)]

# add the predictions to the dataframe
df_clean.loc[~df_clean['movieId'].isin(df_clean[df_clean['userId'] == user_id]['movieId']), 'hybrid_score'] = predictions_combined

In [17]:
#sort the movies by their hybrid scores and select the top 5 movies

# get the top 5 movie recommendations based on the hybrid scores
top_5_movies = df_clean[df_clean['userId'] != user_id].sort_values(by='hybrid_score', ascending=False).head(5)['title']
print(top_5_movies)

96369     Wolf of Wall Street, The (2013)
5444                 Prophecy, The (1995)
53543                    Key Largo (1948)
33854              Assignment, The (1997)
100785                  Deadpool 2 (2018)
Name: title, dtype: object


# Recommendation for top 5

In [23]:
# get the top 5 movie recommendations based on the hybrid scores
top_5_movies = df_clean2[df_clean2['userId'] != user_id].sort_values(by='hybrid_score', ascending=False).head(5)

# iterate over the top 5 recommendations and print the actual rating and predicted rating for each movie
for index, row in top_5_movies.iterrows():
    movie_title = row['title']
    actual_rating = row['rating']
    predicted_rating = row['hybrid_score']
    print(f"Movie: {movie_title}")
    print(f"Actual rating: {actual_rating}")
    print(f"Predicted rating: {predicted_rating}")
    print() 

NameError: name 'df_clean2' is not defined